**A PET Scan Simulator**

By: Nicholas Trigger and James Bradley


Using Ideas from:
    Mahoney, D., Huang, S. C., Ricci, A. R., Mazziotta, J. C., Carson, R. E., Hoffman, E. J., & Phelps, M. E. (1987). A Realistic Computer-Simulated Brain Phantom for Evaluation of PET Charactenstics. IEEE transactions on medical imaging, 6(3), 250–257. https://doi.org/10.1109/TMI.1987.4307834

In [1]:
# RUN ME!

%matplotlib widget
from ipywidgets import (
    AppLayout,
    FloatSlider,
    IntSlider,
    VBox,
    Output,
    interactive,
    Accordion,
    Checkbox,
    Dropdown,
    IntRangeSlider,
    HBox,
    VBox,
    HTML,
    Layout,
    Label, 
    Button,
    widgets
)
import matplotlib.pyplot as plt
from IPython.display import display
import numpy as np
from Helpers.image_to_phantom import img_to_phantom, get_image_info, pet_sim
import plotly.graph_objects as go
from PIL import Image
import scipy.io
from scipy import signal
#!pip install scikit-image
from skimage.transform import radon, iradon
import tqdm as tqdm

plt.ioff()
plt.close('all')

with open("Loading.gif", "rb") as f:
    spinner = widgets.Image(
        value=f.read(),
        format='gif',
        width=100,
        height=100,
    )

# blur slider
blur_slider = IntSlider(value=7, min=0, max=60, description="Resolution:", continuous_update=False)
skull_thickness_slider = IntSlider(value=15, min=0, max=40, description="Skull Thickness:", continuous_update=False)

# Create sliders for Fluence, CT Values and PET Values
fluence_slider = IntSlider(value=10, min=1, max=100, description="Fluence:", continuous_update=False)
c_CSF_slider = FloatSlider(value=1, min=0, max=15, step=0.1, description="CSF ρ:", continuous_update=False)
c_WM_slider = FloatSlider(value=2, min=0, max=15, step=0.1, description="WM ρ:", continuous_update=False)
c_GM_slider = FloatSlider(value=8, min=0, max=15, step=0.1, description="GM ρ:", continuous_update=False)
c_tumor_slider = FloatSlider(value=2, min=0, max=15, step=0.1, description="Tumor ρ:", continuous_update=False)
c_skull_slider = FloatSlider(value=15, min=0, max=30, step=0.1, description="Skull ρ:", continuous_update=False)

p_CSF_slider = FloatSlider(value=0.2, min=0, max=15, step=0.1, description="CSF Metabolic Activity:", continuous_update=False)
p_WM_slider = FloatSlider(value=2.0, min=0, max=15, step=0.1, description="WM Metabolic Activity:", continuous_update=False)
p_GM_slider = FloatSlider(value=3.5, min=0, max=15, step=0.1, description="GM Metabolic Activity:", continuous_update=False)
p_tumor_slider = FloatSlider(value=12.0, min=0, max=15, step=0.1, description="Tumor Metabolic Activity:", continuous_update=False)
p_skull_slider = FloatSlider(value=0.01, min=0, max=2, step=0.01, description="Skull Metabolic Activity:", continuous_update=False)
p_decay_slider = FloatSlider(value=0.1, min=0, max=1, step=0.01, description="Decay Rate:", continuous_update=False)
p_exposure_slider = IntSlider(value=150, min=0, max=600, description="Exposure Time:", continuous_update=False)

# Checkbox for tumor presence and tumor location
tumor_check = Checkbox(value=True, description="Tumor Present", disabled=False, indent=False)
tumor_location_x = IntSlider(value=522, min=0, max=640, step=1, description="Location X:", continuous_update=False)
tumor_location_y = IntSlider(value=220, min=0, max=840, step=1, description="Location Y:", continuous_update=False)
tumor_width_x = FloatSlider(value=20, min=0, max=50, step=0.1, description="Width X:", continuous_update=False)
tumor_width_y = FloatSlider(value=30, min=0, max=50, step=0.1, description="Width Y:", continuous_update=False)

# Button to enable selecting tumor location faster
place_tumor_btn = Button(
    description="Place Tumor",
    button_style='primary'
)

current_fig = None
placement_active = False
current_cid = None

def on_place_tumor_click(b):
    global placement_active, current_cid, current_fig
    if not placement_active:
        place_tumor_btn.description = "Click Plot"
        place_tumor_btn.button_style = 'success'
        # Attach handler to most recent figure
        if current_fig:
            if current_cid:
                current_fig.canvas.mpl_disconnect(current_cid)
            current_cid = current_fig.canvas.mpl_connect('button_press_event', on_plot_click)
            placement_active = True
    else:
        place_tumor_btn.description = "Place Tumor"
        place_tumor_btn.button_style = 'primary'
        if current_cid and current_fig:
            current_fig.canvas.mpl_disconnect(current_cid)
        placement_active = False
        
def on_plot_click(event):
    global placement_active, current_cid, current_fig
    if event.inaxes:
        tumor_location_x.value = int(event.xdata)
        tumor_location_y.value = int(event.ydata)
        tumor_check.value = True
        tumor_location_x.notify_change({'name': 'value', 'new': tumor_location_x.value})
        tumor_location_y.notify_change({'name': 'value', 'new': tumor_location_y.value})
    # Always cleanup
    place_tumor_btn.description = "Place Tumor"
    place_tumor_btn.button_style = 'primary'
    if current_cid and current_fig:
        current_fig.canvas.mpl_disconnect(current_cid)
    placement_active = False

place_tumor_btn.on_click(on_place_tumor_click)

# Widgets for selecting image and output image
imageInfo = get_image_info()

imageSelector = Dropdown(options=imageInfo.keys(), description="Input Image:")
imageOutputSelect = Dropdown(options=[('None', None), ('Original', 'O'), ('CT Phantom', 'G'), ('PET Phantom', 'PP'), ('CT', 'C'), ('PET', 'P'), ('CT + PET', 'CP'), ('CT Sinogram', 'CS'), ('PET Sinogram', 'PS'), ('Poster', 'Po')], description="Output Image:")

# Output widgets
output = Output(layout=Layout(flex_flow = 'row wrap', align_items = 'flex-start', width='100%', height='100%', overflow='hidden')) #plot
logs = Output(layout=Layout(flex_flow = 'row wrap', align_items = 'flex-start', width='100%', height='100%')) #logs (print statements)

prev_query = None

def run_simulation(fluence, c_CSF, c_WM, c_GM, c_tumor, p_CSF, p_WM, p_GM, p_tumor, blur, tumor_present, tumor_location_x, tumor_location_y, tumor_width_x, tumor_width_y, image, outputType, decay_rate, exposure_time, skull_thickness, c_Skull, p_skull):
    
    with output:
        output.clear_output(wait=True)
        display(spinner)

    img_path = imageInfo[image][0]
    whitematter_color = imageInfo[image][1]
    graymatter_color = imageInfo[image][2]

    with logs:
        logs.clear_output(wait=True)
        phantoms, query = img_to_phantom(
            img_path, image, whitematter_color, graymatter_color, 
            0, c_WM, c_GM, c_CSF, p_WM, p_GM, p_CSF, c_Skull, p_skull,
            tumor_params=[tumor_location_x, tumor_location_y, tumor_width_x, 
                        tumor_width_y, c_tumor, p_tumor] if tumor_present else None,
            tolerance_pct=15, brain_bound_padding=40, blur_radius=blur, dbg=False, skull_thickness=skull_thickness,
            fileloc="./BrainPhantoms/"
        )

    ct_phantom = phantoms[0]
    pet_phantom = phantoms[1]

    with output:
        output.clear_output(wait=True)

        fig, ax = plt.subplots()

        # Update plot based on output type
        match outputType:
            case None:
                pass
            case 'O':  # Original
                image_data = plt.imread(img_path)
                cmap = None
            case 'G':  # Generated Phantom
                image_data = ct_phantom
                cmap = 'gray'
            case 'PP':  # PET Phantom
                image_data = pet_phantom
                cmap = 'gray'
            case 'C':  # CT Simulation
                ct_sinogram = radon(ct_phantom)
                ct_simulation = iradon(ct_sinogram, filter_name="hamming", circle=True)
                image_data = ct_simulation
                cmap = 'gray'
            case 'P': # PET Simulation
                pet_sinogram, pet_image = pet_sim(pet_phantom, decay_rate, fluence, exposure_time)
                image_data = pet_image
                cmap = 'hot'
            case 'CP':
                ct_sinogram = radon(ct_phantom)
                ct_simulation = iradon(ct_sinogram, filter_name="hamming", circle=True)
                
                pet_sinogram, pet_image = pet_sim(pet_phantom, decay_rate, fluence, exposure_time)
                
                ax.set_axis_off()
                im = ax.imshow(pet_image, cmap='hot', alpha=0.5)
                im = ax.imshow(ct_simulation, cmap='gray', alpha=0.5)
            case 'CS':  # CT Sinogram
                ct_sinogram = radon(ct_phantom)
                image_data = ct_sinogram
                cmap = 'gray'
            case 'PS':  # PET Sinogram
                pet_sinogram, pet_image = pet_sim(pet_phantom, decay_rate, fluence, exposure_time)
                image_data = pet_sinogram
                cmap = 'hot'
            case 'Po':
                image_data = plt.imread("./poster.jpg")
                cmap = None
            case _:
                raise ValueError(f"Invalid output type: {outputType}")

        if outputType != 'CP':
            ax.set_axis_off()
            im = ax.imshow(image_data, cmap=cmap)
        if outputType in ['CS', 'PS']:
            ax.set_xlabel('Angle (degrees)')
            ax.set_xticks(np.arange(0, 180, 45))
            ax.get_yaxis().set_visible(False)
        else:
            ax.xaxis.set_visible(False)
            ax.yaxis.set_visible(False)

        fig.canvas.header_visible = False
        fig.canvas.toolbar_visible = True
        plt.tight_layout()
        display(fig.canvas)
        global current_fig, current_cid, placement_active
        current_fig = fig

        if placement_active:
            if current_cid:
                current_fig.canvas.mpl_disconnect(current_cid)
            current_cid = current_fig.canvas.mpl_connect('button_press_event', on_plot_click)


    with logs:
        logs.clear_output(wait=True)
        print(f"Processing: {img_path}")
        print(f"Output type: {outputType}")
        print(f"Query parameters: {query}")
        print(f"Size: {image_data.shape}")

# Create an interactive widget
interactiveWindow = interactive(
    run_simulation,
    fluence=fluence_slider,
    c_CSF = c_CSF_slider,
    c_WM = c_WM_slider,
    c_GM = c_GM_slider,
    c_tumor = c_tumor_slider,
    p_CSF = p_CSF_slider,
    p_WM = p_WM_slider,
    p_GM = p_GM_slider,
    p_tumor = p_tumor_slider,
    blur = blur_slider,
    tumor_present = tumor_check,
    tumor_location_x = tumor_location_x,
    tumor_location_y = tumor_location_y,
    image = imageSelector,
    tumor_width_x = tumor_width_x,
    tumor_width_y = tumor_width_y,
    info = imageInfo,
    outputType=imageOutputSelect,
    decay_rate = p_decay_slider,
    exposure_time = p_exposure_slider,
    c_Skull = c_skull_slider,
    p_skull = p_skull_slider,
    skull_thickness = skull_thickness_slider,
)

# Header and footer
headerMsg = "CT and PET Phantom Generator \n and Simulator"
header_fmt = HTML(f"<h1 style='margin-top: 10px;margin-bottom: 45px;text-align: center;font-family: sans-serif;font-size: 2.7rem;letter-spacing: 0.15rem;text-transform: uppercase;color: #fff;text-shadow: -4px 4px #ef3550,-8px 8px #f48fb1,-12px 12px #7e57c2,-16px 16px #2196f3,-20px 20px #26c6da,-24px 24px #43a047,-28px 28px #eeff41,-32px 32px #f9a825,-36px 36px #ff5722;'>{headerMsg}</h1>")
spacer = HTML("<hr style='margin: 2px; border: 2px solid black; border-radius: 5px;'>")
header = VBox([header_fmt, spacer])

footerMsg = "By Nick and James"
footer_fmt = HTML(f"<h2 style='margin-top: 25px;margin-bottom: 0px;text-align: center;font-family: sans-serif;font-size: 1rem;letter-spacing: 0.15rem;text-transform: uppercase;color: #000'>{footerMsg}</h2>")
footer = VBox([spacer, footer_fmt])

# Layout the sliders and plot
CTbox = VBox([c_CSF_slider, c_WM_slider, c_GM_slider, c_tumor_slider, c_skull_slider])
PETbox = VBox([p_CSF_slider, p_WM_slider, p_GM_slider, p_tumor_slider, p_skull_slider, p_decay_slider, p_exposure_slider])
TumorBox = VBox([tumor_check, place_tumor_btn, tumor_location_x, tumor_location_y, tumor_width_x, tumor_width_y])

accordion = Accordion(children=[CTbox, PETbox, TumorBox, logs], layout=Layout(align_items='stretch', justify_content='space-between'))
accordion.set_title(0, 'CT Values')
accordion.set_title(1, 'PET Values')
accordion.set_title(2, 'Tumor Values')
accordion.set_title(3, 'Logs')

menu = VBox([imageSelector, imageOutputSelect, blur_slider, skull_thickness_slider, accordion], layout=Layout(align_items='flex-start', width='100%', overflow='hidden'))
AppLayout(
    header=header,
    center=HBox([output, menu], layout=Layout(width='100%', height='100%', align_items='stretch', justify_content='space-between', overflow='hidden')),
    footer=footer,
    layout=Layout(width='100%', height='100%', align_items='stretch')
)

AppLayout(children=(VBox(children=(HTML(value="<h1 style='margin-top: 10px;margin-bottom: 45px;text-align: cen…

KeyError: 'type'